In [8]:
from PIL import Image
import pandas as pd
import os, cv2
import numpy as np

In [ ]:
!pip install Ninja

In [ ]:
!git clone https://github.com/PeikeLi/Self-Correction-Human-Parsing.git

In [ ]:
!git clone https://github.com/Healthcare-Robotics/bodies-at-rest.git

In [ ]:
!mkdir Self-Correction-Human-Parsing-Segmented-Dataset
!mkdir Self-Correction-Human-Parsing-Dataset

In [13]:
%cd Self-Correction-Human-Parsing

/content/Self-Correction-Human-Parsing


In [14]:
!mkdir input
!mkdir output
!mkdir dataset

In [15]:
%cd /content/bodies-at-rest/PressurePose

/content/bodies-at-rest/PressurePose


In [ ]:
!chmod +x download_real.sh
!bash download_real.sh

In [18]:
object = pd.read_pickle(r'/content/bodies-at-rest/data_BR/real/S103/prescribed.p')
print(len(object['RGB']))
print(object['RGB'][0].shape)

48
(880, 440, 3)


In [19]:
path = "/content/bodies-at-rest/data_BR/real/"
folders = os.listdir(path)
w, h = 880, 440

for i, folder in enumerate(folders):
  object = pd.read_pickle(r'/content/bodies-at-rest/data_BR/real/'+ folder +'/prescribed.p')
  for j in range(0, len(object['RGB'])):
    rgb_img = np.zeros((h, w, 3), dtype=np.uint8)
    rgb_img = object['RGB'][j]
    rgb_img = Image.fromarray(rgb_img, 'RGB')
    rgb_img.save(f'/content/Self-Correction-Human-Parsing-Dataset/{i}{j}.jpg')

In [20]:
!python '/content/Self-Correction-Human-Parsing/simple_extractor.py' --dataset 'pascal' --model-restore '/content/drive/MyDrive/models/exp-schp-201908270938-pascal-person-part.pth' --input-dir '/content/Self-Correction-Human-Parsing-Dataset' --output-dir '/content/Self-Correction-Human-Parsing-Segmented-Dataset'

Evaluating total class number 7 with ['Background', 'Head', 'Torso', 'Upper Arms', 'Lower Arms', 'Upper Legs', 'Lower Legs']
100% 914/914 [03:59<00:00,  3.82it/s]


In [21]:
path = "/content/Self-Correction-Human-Parsing-Segmented-Dataset/"
segmented_images = os.listdir(path)

len(segmented_images)

914

In [22]:
%cd /content/Self-Correction-Human-Parsing/dataset/
!mkdir train_images
!mkdir train_segmentations
!mkdir val_images
!mkdir val_segmentations

/content/Self-Correction-Human-Parsing/dataset


In [23]:
train_id = open('train_id.txt', 'w')
val_id = open('val_id.txt', 'w')

for i ,image in enumerate(segmented_images):
  img = cv2.imread(os.path.join(path, image))
  img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  img_gray[img_gray > 0] = 255
  img_gray[img_gray == 0] = 0
  image_name = image.split('.')[0]
  if i < 800:
    cv2.imwrite(f"train_images/{image_name}.jpg", img_gray)
    train_id.write(f"{image_name}")
    train_id.write('\n')
  else:
    cv2.imwrite(f"val_images/{image_name}.jpg", img_gray)
    val_id.write(f"{image_name}")
    val_id.write('\n')

train_id.close()
val_id.close()

In [24]:
path = "/content/Self-Correction-Human-Parsing-Segmented-Dataset/"
segmented_images = os.listdir(path)

for i, image in enumerate(segmented_images):
  img_rgb = cv2.imread(os.path.join(path, image))
  img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

  img_gray[img_gray == 0] = 0
  img_gray[img_gray == 15] = 1
  img_gray[img_gray == 38] = 2
  img_gray[img_gray == 53] = 3
  img_gray[img_gray == 75] = 4
  img_gray[img_gray == 90] = 5
  img_gray[img_gray == 113] = 6

  if i < 800:
    cv2.imwrite(f"train_segmentations/{image}", img_gray)
  else:
    cv2.imwrite(f"val_segmentations/{image}", img_gray)

In [25]:
%cd /content/Self-Correction-Human-Parsing/
!mkdir pretrain_model

/content/Self-Correction-Human-Parsing


In [26]:
import gdown

url = "http://sceneparsing.csail.mit.edu/model/pretrained_resnet/resnet101-imagenet.pth"
out = "/content/Self-Correction-Human-Parsing/pretrain_model/resnet101-imagenet.pth"
gdown.download(url, out) 

Downloading...
From: http://sceneparsing.csail.mit.edu/model/pretrained_resnet/resnet101-imagenet.pth
To: /content/Self-Correction-Human-Parsing/pretrain_model/resnet101-imagenet.pth
100%|██████████| 179M/179M [00:02<00:00, 62.9MB/s]


'/content/Self-Correction-Human-Parsing/pretrain_model/resnet101-imagenet.pth'

In [ ]:
!python '/content/Self-Correction-Human-Parsing/train.py' --batch-size 8 --data-dir '/content/Self-Correction-Human-Parsing/dataset' --epochs 10 --num-classes 7